In [1]:
from keras.models import load_model
from keras import regularizers
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Flatten, Dropout
from keras.layers.merge import Dot, Concatenate
from keras.utils import np_utils, plot_model
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split
import numba as nb


Using TensorFlow backend.


In [2]:
class read_train_model():
    
    def __init__(self,data_path):
        self.data_path=data_path
        self.df_in=pd.DataFrame()
        self.df_train=pd.DataFrame()
        self.df_test=pd.DataFrame()
        self.n_users=0
        self.n_posts=0
        self.max_user=0
        self.max_post=0
        
    def read_data(self):
        self.df_in = pd.read_csv(self.data_path)
        
        self.df_in.loc[self.df_in.Likes>0,'Likes']=1
        self.df_in.loc[self.df_in.Comments>0,'Comments']=1
        self.df_in.loc[self.df_in.Shares>0,'Shares']=1
        self.df_in.loc[self.df_in.Downloads>0,'Downloads']=1
        self.df_in.loc[self.df_in.Views>0,'Views']=1

        #self.df_in['Rating']=self.df_in['Likes']+self.df_in['Comments']+self.df_in['Shares']+self.df_in['Downloads']+self.df_in['Views']
        #self.df_in.drop(['Likes','Comments','Shares','Downloads','Views'],axis=1,inplace=True)
        
        #clip_point=self.df_in.Rating.quantile(.98)
        #indexNames = self.df_in[self.df_in['Rating'] > clip_point].index
        #self.df_in.drop(indexNames , inplace=True)
        #self.df_in.UserId = self.df_in.UserId.astype(np.int16)
        #self.df_in.PostId = self.df_in.PostId.astype(np.int16)

        self.df_train,self.df_test =train_test_split(self.df_in, test_size = 0.1,random_state = 42 )

        self.n_users = len(self.df_in.UserId.unique()) 
        self.n_posts = len(self.df_in.PostId.unique())
        self.max_user,self.max_post=max(self.df_in.UserId),max(self.df_in.PostId)
        

    def define_model(self):
        post_input = Input(shape=[1], name="post-Input")
        post_embedding = Embedding(self.max_post+1,100,  name="post-Embedding")(post_input)
        lp = Dense(100,activation = 'relu',kernel_regularizer=regularizers.l2(0.001),)(post_embedding)
        Dropout(0.4)
        post_vec = Flatten(name="Flatten-post")(lp)

        user_input = Input(shape=[1], name="User-Input")
        user_embedding = Embedding(self.max_user+1, 100, name="User-Embedding")(user_input)
        l2 = Dense(100,activation = 'relu',kernel_regularizer=regularizers.l2(0.001))(user_embedding)
        Dropout(0.4)
        user_vec = Flatten(name="Flatten-Users")(l2)

        product_layer = Concatenate(name="Concat",)([post_vec, user_vec])

        fully_connected_layer = Dense(200,activation ='relu')(product_layer)
        fully_connected_layer_2 = Dense(100,activation ='relu')(fully_connected_layer)
        fully_connected_layer_3 = Dense(100,activation ='relu')(fully_connected_layer_2)
        fully_connected_layer_4 = Dense(100,activation ='relu')(fully_connected_layer_3)


        output_connected_layer = Dense(5,activation ='sigmoid')(fully_connected_layer_4)

        model = Model([user_input, post_input],output_connected_layer)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["binary_accuracy"])
        return model
    
    def train_model(self):
        model =self.define_model()
        y_df=self.df_train[['Likes','Comments','Shares','Downloads','Views']]
        y=y_df.values
        history = model.fit([self.df_train.UserId, self.df_train.PostId], y,validation_split=0.1 , epochs= 1, verbose=1)
        model.save('recommender_model.h5')
        return history
    
    def get_model(self):
        model = load_model('recommender_model.h5')
        print('model loaded')
        return model
    


In [3]:
rec_model=read_train_model('/home/gaurav/Desktop/RecEng/ML-DataSet/Wall_Activity_User_Post.csv')
rec_model.read_data()
rec_model.define_model()
#plot_model(rec_model)
rec_model.train_model()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 174008 samples, validate on 19335 samples
Epoch 1/1
174008/174008 [==============================] - 778s 4ms/step - loss: 0.1357 - binary_accuracy: 0.9622 - val_loss: 0.1243 - val_binary_accuracy: 0.9624


In [4]:
df_test=rec_model.df_test
df_test1=df_test.sample(5000)
posts=df_test1.PostId
users=df_test1.UserId

model=rec_model.get_model()
est=model.predict([users,posts])
#est1=np.round(est,0)
#df_test1['Estimates']=est



model loaded


In [12]:
df_test.head()

,UserId,PostId,Likes,Comments,Shares,Downloads,Views
183899,98135,2899,0.0,0,0,0.0,1
15771,53721,2929,1.0,0,0,0.0,0
39827,82595,2471,0.0,0,0,0.0,1
132080,91669,2767,0.0,0,0,0.0,1
207959,75661,2863,0.0,0,0,0.0,1


In [14]:
est1=np.round(est,0)

In [43]:
df_test1['estlike']=est1[:,0]
df_test1['estcomment']=est1[:,1]
df_test1['estshare']=est1[:,2]
df_test1['estdownload']=est1[:,3]
df_test1['estview']=est1[:,4]

In [44]:
df_test1

,UserId,PostId,Likes,Comments,Shares,Downloads,Views,estlike,estcomment,estshare,estdownload,estview
131122,83229,2767,0.0,0,0,0.0,1,0.0,0.0,0.0,0.0,1.0
125324,38208,2752,0.0,0,0,0.0,1,0.0,0.0,0.0,0.0,1.0
46993,76278,2506,0.0,0,0,0.0,1,0.0,0.0,0.0,0.0,1.0
114950,85669,2687,0.0,0,0,0.0,1,0.0,0.0,0.0,0.0,1.0
160447,52487,2760,0.0,0,0,0.0,1,0.0,0.0,0.0,0.0,1.0
119056,32634,2645,0.0,0,0,0.0,1,0.0,0.0,0.0,0.0,1.0
20542,46724,2672,0.0,1,0,0.0,1,0.0,0.0,0.0,0.0,1.0
111796,40863,2714,0.0,0,0,0.0,1,0.0,0.0,0.0,0.0,1.0
140157,69378,2791,0.0,0,0,0.0,1,0.0,0.0,0.0,0.0,1.0
166431,59984,2841,0.0,0,0,0.0,1,0.0,0.0,0.0,0.0,1.0


In [47]:
df_test1.groupby('estview').size()

estview
1.0    5000
dtype: int64

In [48]:
df_test1.groupby('Views').size()

Views
0      90
1    4910
dtype: int64

In [50]:
df=rec_model.df_in
df['Rating']=df['Likes']+df['Comments']+df['Shares']+df['Downloads']+df['Views']
df.drop(['Likes','Comments','Shares','Downloads','Views'],axis=1,inplace=True)
df.groupby('Rating').size()

Rating
0.0        89
1.0    187836
2.0     20032
3.0      6185
4.0       648
5.0        36
dtype: int64